# Modular Prompt Library Builder  
Design reusable prompt “blocks”, render complete prompts with **Jinja2**, track token counts, and save to a local JSON library.

In [ ]:
!pip -q install jinja2 openai tiktoken textstat

In [ ]:
import os, json, uuid, openai, tiktoken
from jinja2 import Template
import ipywidgets as w
from IPython.display import display, Markdown

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

# File to store prompt blocks
LIB_PATH = 'prompt_blocks.json'
try:
    with open(LIB_PATH) as fp:
        library = json.load(fp)
except FileNotFoundError:
    library = {}

def save_library():
    with open(LIB_PATH, 'w') as fp:
        json.dump(library, fp, indent=2)

# --- Widgets ---
block_name = w.Text(value='', description='Block name:')
block_content = w.Textarea(value='{{ instruction }}', layout=w.Layout(width='100%', height='120px'))
add_btn = w.Button(description='Add / Update Block')
blocks_dropdown = w.Dropdown(options=['--select--'] + list(library.keys()), description='Blocks')

render_tmpl = w.Textarea(value='{{ library.intro }}\n\n{{ library.task }}', 
                         layout=w.Layout(width='100%', height='120px'),
                         description='Template:')

context_box = w.Textarea(value='instruction=Summarize the article below in 3 bullet points',
                         layout=w.Layout(width='100%', height='80px'),
                         description='Context Vars:')

render_btn = w.Button(description='Render & Run 👉')
out = w.Output()

# --- Callbacks ---
def add_block(b):
    library[block_name.value.strip()] = block_content.value
    save_library()
    blocks_dropdown.options = ['--select--'] + list(library.keys())

def on_select(change):
    sel = change['new']
    if sel and sel != '--select--':
        block_name.value = sel
        block_content.value = library[sel]

add_btn.on_click(add_block)
blocks_dropdown.observe(on_select, names='value')

def render_and_run(b):
    with out:
        out.clear_output()
        # parse context vars
        vars_dict = {}
        for line in context_box.value.splitlines():
            if '=' in line:
                k, v = line.split('=',1)
                vars_dict[k.strip()] = v.strip()
        tmpl = Template(render_tmpl.value)
        rendered = tmpl.render(library=library, **vars_dict)
        print("🔧 Rendered Prompt:\n", rendered)
        # token count
        enc = tiktoken.encoding_for_model('gpt-4o-mini')
        n_tokens = len(enc.encode(rendered))
        print(f"\nToken count: {n_tokens}")
        # optional model call
        try:
            resp = openai.ChatCompletion.create(
                model='gpt-4o-mini',
                messages=[{'role':'user','content': rendered}],
                temperature=0.7,
                max_tokens=256
            )
            display(Markdown("### ✨ Model Response\n" + resp.choices[0].message.content))
        except Exception as e:
            print("Model call error:", e)

render_btn.on_click(render_and_run)

ui = w.VBox([
    w.HBox([block_name, add_btn]),
    block_content,
    blocks_dropdown,
    w.Label("🧩 Use blocks in Jinja template below (`library.block_name`)"),
    render_tmpl,
    context_box,
    render_btn,
    out
])
display(ui)
print("Create blocks, compose templates, and track token counts!")


---  
### Workflow Ideas  
1. Store persona blocks, task instructions, and output schemas separately.  
2. Version your JSON library with Git for collaboration.  
3. Use token counts to stay within context limits.  
4. Combine with Day‑1 decoding controls for full pipeline experiments.